<a href="https://colab.research.google.com/github/Kepners/ChopOnions/blob/main/Trndzo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Block 1: Install Required Libraries
!pip install openai==0.27.8 praw==7.7.0 pytrends==4.9.0 feedparser==6.0.10 requests cachetools==5.3.1 prettytable==3.7.0 textblob==0.17.1 python-dotenv==0.21.0 fuzzywuzzy==0.18.0 python-Levenshtein==0.20.9


In [24]:
# Block 2: Mount Google Drive and Load Environment Variables
from google.colab import drive
import os
from dotenv import load_dotenv

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your .env file in Google Drive
dotenv_path = '/content/drive/MyDrive/Secrets/.env'

# Load the environment variables from the .env file
load_dotenv(dotenv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [25]:
# Block 3: Configure Logging
import logging

# Configure Logging to write to a log file and suppress console output
log_file = 'app.log'
logging.basicConfig(
    level=logging.ERROR,  # Only log ERROR and CRITICAL
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(open(os.devnull, 'w'))  # Suppress console output
    ]
)


In [26]:
# Block 4: Download NLTK Data Silently
import nltk
import sys
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('brown')
    nltk.download('wordnet')
    nltk.download('movie_reviews')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [27]:
# Block 5: Create Utils Directory
import os

utils_dir = 'utils'
if not os.path.exists(utils_dir):
    os.makedirs(utils_dir)


In [28]:
# Block 6: Create data_processing.py
data_processing_code = """
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from textblob import TextBlob

def extract_keywords(text, max_keywords=10):
    \"""
    Extracts up to `max_keywords` nouns from the input `text`.
    \"""
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
    # Get part-of-speech tags
    tagged_words = pos_tag(filtered_words)
    # Keep nouns and proper nouns
    keywords = [word for word, pos in tagged_words if pos.startswith('NN')]
    # Limit the number of keywords
    return keywords[:max_keywords]

def analyze_sentiment(text):
    \"""
    Analyzes the sentiment of the given text and returns it.
    \"""
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0.1:
        return 'Positive'
    elif polarity < -0.1:
        return 'Negative'
    else:
        return 'Neutral'
"""
with open(os.path.join('utils', 'data_processing.py'), 'w') as file:
    file.write(data_processing_code)


In [29]:
# Block 7: Initialize API Clients and Configure Logging
import praw
import openai
import logging
import warnings
from utils.data_processing import extract_keywords, analyze_sentiment
from cachetools import TTLCache, cached
from pytrends.request import TrendReq
import feedparser
import requests
import time
import os
from fuzzywuzzy import process

# Suppress PRAW warnings about asynchronous environments
warnings.filterwarnings("ignore", category=UserWarning, module='praw')
logging.getLogger('praw').setLevel(logging.CRITICAL)

# Initialize Reddit API using PRAW
try:
    reddit = praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
        user_agent=os.getenv('REDDIT_USER_AGENT', 'script:TrendingTopicsScript:1.0 (by u/yourusername)')
    )
except Exception as e:
    logging.error(f"Error initializing Reddit API: {e}")

# Initialize OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')

# Initialize Pexels API key
PEXELS_API_KEY = os.getenv('PEXELS_API_KEY')

# Initialize Shutterstock Access Token
SHUTTERSTOCK_ACCESS_TOKEN = os.getenv('SHUTTERSTOCK_ACCESS_TOKEN')

# Define a list of countries with their codes, including common abbreviations
available_countries = {
    'United States': 'US',
    'US': 'US',
    'USA': 'US',
    'India': 'IN',
    'Canada': 'CA',
    'United Kingdom': 'GB',
    'UK': 'GB',
    'Great Britain': 'GB',
    'Australia': 'AU',
    'Germany': 'DE',
    'France': 'FR',
    'Brazil': 'BR',
    'Mexico': 'MX',
    'Japan': 'JP',
    'Russia': 'RU',
    'South Korea': 'KR',
    'Korea': 'KR',
    'Italy': 'IT',
    'Spain': 'ES',
    'Netherlands': 'NL',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'Austria': 'AT',
    'Belgium': 'BE',
    'New Zealand': 'NZ'
}

# Mapping from country codes to PyTrends 'pn' parameter for trending searches
# For realtime_trending_searches, the acceptable 'pn' values are specific
country_code_to_pn_realtime = {
    'US': 'US',
    'IN': 'IN',
    'GB': 'GB',
    'CA': 'CA',
    'AU': 'AU',
    'DE': 'DE',
    'FR': 'FR',
    'BR': 'BR',
    'MX': 'MX',
    'JP': 'JP',
    'RU': 'RU',
    'KR': 'KR'  # South Korea
}

# For trending_searches, the acceptable 'pn' values are:
country_code_to_pn_daily = {
    'united_states': 'united_states',
    'us': 'united_states',
    'india': 'india',
    'united_kingdom': 'united_kingdom',
    'uk': 'united_kingdom',
    'gb': 'united_kingdom',
    'canada': 'canada',
    'australia': 'australia',
    'germany': 'germany',
    'france': 'france',
    'brazil': 'brazil'
    # Note: Limited countries are available for trending_searches
}


In [30]:
# Block 8: Define Supporting Functions
from fuzzywuzzy import process
import re
import textwrap

def get_matching_country(input_country, available_countries):
    # Combine country names and codes
    country_list = list(available_countries.keys())
    # Use fuzzy matching to find the best match
    match, score = process.extractOne(input_country, country_list)
    if score >= 70:  # Threshold can be adjusted
        return match
    else:
        return None

def fetch_trending_topics_pytrends(selected_country_code, time_range):
    try:
        pytrends = TrendReq(hl='en-US', tz=360)
        if time_range == '4h':
            # Realtime trending searches
            pn = country_code_to_pn_realtime.get(selected_country_code)
            if not pn:
                logging.error(f"Realtime trending searches not available for the selected country.")
                return []
            df = pytrends.realtime_trending_searches(pn=pn)
        else:
            # Daily trending searches
            pn = country_code_to_pn_daily.get(selected_country_code.lower())
            if not pn:
                logging.error(f"Daily trending searches not available for the selected country.")
                return []
            df = pytrends.trending_searches(pn=pn)
        if df.empty:
            return []
        trending_topics = []
        for index, row in df.iterrows():
            title = row.iloc[0]  # Use .iloc[0] instead of row[0]
            # Generate a brief reason why the topic is trending
            reason = generate_trend_reason(title)
            trending_topics.append({
                'title': title,
                'description': reason,  # Brief reason why it's trending
                'sv': 'High',       # Placeholder for Search Volume
                'change': 'Up',     # Placeholder for Change
                'started': 'Recently'   # Placeholder
            })
        return trending_topics[:10]  # Limit to top 10
    except Exception as e:
        logging.error(f"Error fetching Google Trends data via PyTrends: {e}")
        return []

def generate_trend_reason(topic):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Use 'gpt-4' model
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that provides brief reasons why a topic is trending. Your responses should be concise, in brackets, and two to four words long."
                },
                {
                    "role": "user",
                    "content": f"Why is '{topic}' trending? Provide a brief reason in brackets, two to four words long."
                }
            ],
            max_tokens=20,
            temperature=0.5,
        )
        reason = response.choices[0].message.content.strip()
        return reason
    except Exception as e:
        logging.error(f"Error generating trend reason: {e}")
        return "[Trending Topic]"

def generate_summary(content):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Use 'gpt-4' model
            messages=[
                {
                    "role": "system",
                    "content": "You are a concise summarizer. Provide a clear and brief two-sentence summary of the following content."
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            max_tokens=150,  # Increased max_tokens for longer summaries
            temperature=0.5,
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        logging.error(f"Error generating summary: {e}")
        return "No summary available."

def search_subreddits_for_topic(topic, limit=4):
    """
    Searches for subreddits related to the topic using individual entities.
    """
    try:
        # Split the topic by commas to get individual entities
        entities = [entity.strip() for entity in topic.split(',')]
        related_subreddits = {}
        for entity in entities:
            # Use each entity as a query
            subreddits_found = reddit.subreddits.search(query=entity, limit=limit)
            for subreddit in subreddits_found:
                # Fetch subreddit details
                try:
                    subreddit_details = reddit.subreddit(subreddit.display_name)
                    # Check if subreddit is accessible and appropriate
                    if subreddit_details.over18 or subreddit_details.subreddit_type in ['private', 'restricted']:
                        continue
                    # Add subreddit if not already in the list
                    if subreddit.display_name.lower() not in related_subreddits:
                        related_subreddits[subreddit.display_name.lower()] = {
                            'name': subreddit.display_name,
                            'title': subreddit.title,
                            'description': subreddit.public_description or ''
                        }
                    # Break if we've reached the limit
                    if len(related_subreddits) >= limit:
                        break
                except Exception as inner_e:
                    logging.error(f"Error accessing subreddit '{subreddit.display_name}': {inner_e}")
                    continue
            if len(related_subreddits) >= limit:
                break  # Stop searching if we've found enough subreddits
        return list(related_subreddits.values())
    except Exception as e:
        logging.error(f"Error searching for subreddits: {e}")
        return []

def fetch_related_news(topic, limit=5):
    """
    Fetches related news articles using Google News RSS feed.
    """
    try:
        feed_url = f"https://news.google.com/rss/search?q={requests.utils.quote(topic)}&hl=en-US&gl=US&ceid=US:en"
        feed = feedparser.parse(feed_url)
        related_stories = []
        for entry in feed.entries[:limit]:
            title = entry.title
            summary = entry.summary if 'summary' in entry else ''
            # Remove HTML tags from summary
            summary = re.sub('<[^<]+?>', '', summary)
            if not summary.strip():
                # Generate summary using OpenAI if empty
                summary = generate_summary(title)
            link = entry.link
            related_stories.append({'title': title, 'summary': summary, 'url': link})
        return related_stories
    except Exception as e:
        logging.error(f"Error fetching related news: {e}")
        return []

@cached(TTLCache(maxsize=1000, ttl=86400))
def check_stock_media_availability(topic):
    # Extract keywords from the topic
    keywords = extract_keywords(topic)
    if not keywords:
        logging.error(f"No valid keywords extracted for topic: {topic}")
        return 0
    # Join keywords into a query string
    query = ' '.join(keywords)[:100]
    total_results = 0
    # Check Pexels
    total_results += check_pexels_media(query)
    # Check Shutterstock
    total_results += check_shutterstock_media(query)
    return total_results

def check_pexels_media(query):
    headers = {
        'Authorization': PEXELS_API_KEY,
        'User-Agent': 'TrendingTopicsScript'
    }
    params = {
        'query': query,
        'per_page': 1,
        'page': 1
    }
    try:
        response = requests.get('https://api.pexels.com/v1/search', headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get('total_results', 0)
        else:
            logging.error(f"Pexels API error {response.status_code}: {response.text}")
            return 0
    except requests.exceptions.RequestException as e:
        logging.error(f"Error checking Pexels for '{query}': {e}")
        return 0

def check_shutterstock_media(query):
    headers = {
        'Authorization': f'Bearer {SHUTTERSTOCK_ACCESS_TOKEN}',
        'User-Agent': 'TrendingTopicsScript'
    }
    params = {
        'query': query,
        'per_page': 1,
        'page': 1,
        'view': 'minimal'
    }
    try:
        response = requests.get('https://api.shutterstock.com/v2/images/search', headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get('total_count', 0)
        else:
            logging.error(f"Shutterstock API error {response.status_code}: {response.text}")
            return 0
    except requests.exceptions.RequestException as e:
        logging.error(f"Error checking Shutterstock for '{query}': {e}")
        return 0

def generate_script_for_topic(topic, content_summary, style="Normal Script"):
    try:
        style_prompts = {
            "Flashy Script": "Create a flashy, high-energy script with quick cuts, bold visuals, and impactful statements.",
            "Expressive Script": "Compose an expressive script that evokes emotions, using descriptive language and a storytelling approach.",
            "Normal Script": "Write a straightforward script that clearly presents the information in an engaging manner."
        }
        style_prompt = style_prompts.get(style, style_prompts["Normal Script"])
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Use 'gpt-4' model
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a creative scriptwriter for short-form videos. "
                        "Produce engaging, dynamic scripts that are visually compelling and can be represented "
                        "using stock images and videos. The script should be suitable for a 60-second video, "
                        "have a captivating introduction, a clear narrative flow, and a strong conclusion or call-to-action. "
                        "Include specific details about the trend, such as goal scores, player performances, bookings, and other relevant statistics."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"{style_prompt}\n"
                        f"Using the following summary, write a detailed script suitable for a 60-second video:\n\n"
                        f"Summary: {content_summary}\n\n"
                        "Ensure the script includes vivid descriptions and is structured with a beginning, middle, and end. "
                        "Use language that resonates with the target audience, and make sure it's adaptable with widely available stock images and videos."
                    )
                }
            ],
            max_tokens=1500,  # Increased max_tokens for longer scripts
            temperature=0.7,
        )
        script_content = response.choices[0].message.content.strip()
        return script_content
    except Exception as e:
        logging.error(f"Error generating script: {e}")
        return "No script available."

def generate_fan_sentiment_summary(title, sentiment):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Use 'gpt-4' model
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that summarizes fan sentiments from online discussions into a brief overview."
                },
                {
                    "role": "user",
                    "content": f"The following Reddit post has a {sentiment} sentiment:\n\n'{title}'\n\nSummarize this sentiment for inclusion in a video script."
                }
            ],
            max_tokens=200,  # Increased max_tokens for longer summaries
            temperature=0.5,
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        logging.error(f"Error generating fan sentiment summary: {e}")
        return "No summary available."


In [31]:
# Block 9: Define the Main Function
def main():
    from IPython.display import display, Markdown
    from prettytable import PrettyTable
    import textwrap

    # Step 1: User selects the country for Google Trends
    print("Enter the country for Google Trends data (e.g., United States):")
    country_input = input("Country: ").strip()
    matching_country = get_matching_country(country_input, available_countries)
    if not matching_country:
        print("No matching countries found. Exiting.")
        return
    selected_country = matching_country
    selected_country_code = available_countries[selected_country]
    print(f"You selected: {selected_country}")

    # Step 1b: User selects the time range for Google Trends
    print("\nSelect the time range for trending topics:")
    print("\033[1m1.\033[0m Last 4 hours (Realtime)")
    print("\033[1m2.\033[0m Last 24 hours (Daily)")
    time_range_selection = input("Enter the number of the time range you're interested in: ")
    time_range_mapping = {'1': '4h', '2': '24h'}
    time_range = time_range_mapping.get(time_range_selection)
    if not time_range:
        print("Invalid selection. Exiting.")
        return

    # List locations with daily results available
    if time_range == '24h':
        print("\nLocations with daily trending data available:")
        for loc in country_code_to_pn_daily.keys():
            print(f"- {loc.capitalize()}")

    # Step 2: Fetch trending topics for the selected country and time range
    google_trends_topics = fetch_trending_topics_pytrends(selected_country_code, time_range)

    if not google_trends_topics:
        logging.error("No trending topics found using PyTrends.")
        print("No trending topics found for the selected country and time range.")
        return

    # Display the trending topics with additional information in a table
    print(f"\nCurrent Trending Topics in {selected_country}:\n")
    table = PrettyTable()
    table.field_names = ["No.", "Topic", "Reason"]
    table.max_width = 40  # Set maximum width for columns
    for idx, topic in enumerate(google_trends_topics, start=1):
        title = textwrap.fill(topic['title'], width=40)
        reason = textwrap.fill(topic['description'], width=30)
        table.add_row([idx, title, reason])
    print(table)

    # Allow the user to select a topic
    try:
        selected_idx = int(input("Enter the number of the topic you're interested in: "))
        if 1 <= selected_idx <= len(google_trends_topics):
            selected_topic = google_trends_topics[selected_idx - 1]['title']
            print(f"\nYou selected: {selected_topic}")
        else:
            logging.error("Invalid selection. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter a number. Exiting.")
        return

    # Step 3: Fetch related news articles
    related_stories = fetch_related_news(selected_topic, limit=5)
    if related_stories:
        print("\nRelated News Articles:")
        for idx, story in enumerate(related_stories, start=1):
            story_title = textwrap.fill(story['title'], width=70)
            story_summary = textwrap.fill(story['summary'], width=70)
            display(Markdown(f"**{idx}. {story_title}**\n{story_summary}\n"))
    else:
        print("\nNo related news articles found.")

    # Step 4: Search for subreddits related to the topic
    related_subreddits = search_subreddits_for_topic(selected_topic, limit=4)
    if related_subreddits:
        print("\nRelated Subreddits:")
        for idx, sub in enumerate(related_subreddits, start=1):
            sub_name = sub['name']
            sub_title = textwrap.fill(sub['title'], width=50)
            display(Markdown(f"**{idx}. {sub_name}** - *{sub_title}*"))
    else:
        print("\nNo related subreddits found.")

    # Fetch and display top posts from subreddits
    all_posts = []
    if related_subreddits:
        print("\nTop Reddit Posts from Related Subreddits:")
        for sub in related_subreddits:
            subreddit_name = sub['name']
            try:
                subreddit = reddit.subreddit(subreddit_name)
                top_posts = list(subreddit.hot(limit=5))
                for post in top_posts:
                    if post.over_18:
                        continue
                    if post.is_self and len(post.selftext) < 20:
                        continue  # Skip short self-posts
                    all_posts.append({'subreddit': subreddit_name, 'title': post.title, 'score': post.score, 'url': post.url})
            except praw.exceptions.APIException as e:
                logging.error(f"APIException when accessing subreddit '{subreddit_name}': {e}")
            except Exception as e:
                logging.error(f"Error fetching posts from subreddit '{subreddit_name}': {e}")

        if all_posts:
            # Display posts in a table with formatting
            table = PrettyTable()
            table.field_names = ["No.", "Subreddit", "Title", "Upvotes"]
            table.max_width = 40  # Set maximum width for columns
            for idx, post in enumerate(all_posts, start=1):
                title = textwrap.fill(post['title'], width=40)
                table.add_row([idx, post['subreddit'], title, post['score']])
            print(table)
        else:
            print("\nNo suitable Reddit posts found.")
    else:
        print("\nNo Reddit posts to display.")

    # Combine news articles and Reddit posts
    combined_content = []
    for story in related_stories:
        combined_content.append({'type': 'news', 'title': story['title'], 'url': story['url'], 'summary': story.get('summary', '')})
    for post in all_posts:
        combined_content.append({'type': 'reddit', 'title': post['title'], 'url': post['url'], 'score': post['score'], 'summary': ''})

    if not combined_content:
        logging.error("No content available for script generation. Exiting.")
        return

    # Allow user to select content for script generation
    print("\nAvailable Content for Script Generation:")
    table = PrettyTable()
    table.field_names = ["No.", "Type", "Title", "Upvotes"]
    table.max_width = 40  # Set maximum width for columns
    for idx, content in enumerate(combined_content, start=1):
        content_type = "News Article" if content['type'] == 'news' else "Reddit Post"
        upvotes = content.get('score', '')
        title = textwrap.fill(content['title'], width=40)
        table.add_row([idx, content_type, title, upvotes])
    print(table)

    selected_content_input = input("\nEnter the numbers of the items you want to generate scripts for, separated by commas (e.g., 1,3,5): ")
    try:
        selected_indices = [int(i.strip()) - 1 for i in selected_content_input.split(',') if i.strip().isdigit()]
        selected_items = [combined_content[i] for i in selected_indices if 0 <= i < len(combined_content)]
        if not selected_items:
            logging.error("No valid items selected. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter numbers separated by commas. Exiting.")
        return

    # Fetch content and generate summaries for selected items
    print(f"\nFetching content and generating summaries for selected items...")
    for item in selected_items:
        title = item['title']
        if item['type'] == 'reddit':
            # Analyze sentiment and include upvotes
            sentiment = analyze_sentiment(title)
            item['sentiment'] = sentiment
            item['summary'] = generate_fan_sentiment_summary(title, sentiment)
        else:
            # For news articles, use the summary if available
            if not item['summary']:
                item['summary'] = generate_summary(title)

    # Decide Whether to Check Stock Media Availability
    check_media = input("\nDo you want to check for stock media availability for the selected items? (yes/no): ").strip().lower()
    if check_media not in ['yes', 'y']:
        print("\nSkipping stock media availability check.")
        # Proceed to generate scripts without checking
        generate_scripts_from_items(selected_items)
        print("\nScript execution completed.")
        return

    # Continue with stock media availability check
    print("\nChecking stock media availability for the selected items...")
    request_count = 0
    scripts_generated = 0
    for item in selected_items:
        title = item['title']
        summary = item.get('summary', "No summary available.")
        if request_count >= 190:
            logging.error("Approaching API rate limits. Waiting for 60 minutes before continuing...")
            time.sleep(3600)  # Wait for an hour
            request_count = 0
        total_media = check_stock_media_availability(title)
        request_count += 1
        if total_media >= 10:
            # Display summary before generating script
            print(f"\nSummary: {summary}")
            # Generate script for this topic
            generate_script_with_style(title, summary)
            scripts_generated += 1
        else:
            print(f"\nSummary: {summary}")
            print(f"Not enough stock media for topic: {title}\n====\n")
    if scripts_generated == 0:
        logging.error("No topics with sufficient stock media were found.")
    else:
        logging.info(f"Generated {scripts_generated} script(s) based on available stock media.")
    print("\nScript execution completed.")

def generate_scripts_from_items(selected_items):
    for item in selected_items:
        title = item['title']
        summary = item.get('summary', "No summary available.")
        print(f"\nSummary: {summary}")
        generate_script_with_style(title, summary)
        print("====\n")

def generate_script_with_style(topic, summary):
    from IPython.display import display, Markdown
    print("\nSelect a script style:")
    print("\033[1m1.\033[0m Flashy Script")
    print("\033[1m2.\033[0m Expressive Script")
    print("\033[1m3.\033[0m Normal Script")
    style_choice = input("Enter the number of the script style you prefer: ")
    style_mapping = {'1': 'Flashy Script', '2': 'Expressive Script', '3': 'Normal Script'}
    style = style_mapping.get(style_choice, 'Normal Script')
    script = generate_script_for_topic(topic, summary, style=style)
    # Display the script with enhanced formatting
    display(Markdown(f"### Generated {style} for '{topic}':\n\n{script}"))


In [36]:
# Block 10: Run the Main Function
if __name__ == "__main__":
    main()


Enter the country for Google Trends data (e.g., United States):
Country: fr
You selected: France

Select the time range for trending topics:
1. Last 4 hours (Realtime)
2. Last 24 hours (Daily)
Enter the number of the time range you're interested in: 1

Current Trending Topics in France:

+-----+------------------------------------------+--------------------------------+
| No. |                  Topic                   |             Reason             |
+-----+------------------------------------------+--------------------------------+
|  1  | FC Nantes, Olympique de Marseille, Ligue |     (Football match event)     |
|     |         1, La Beaujoire Stadium          |                                |
|  2  | Bob Sinclar, 20H30 Le Dimanche, Laurent  |      (TV show appearance)      |
|     | Delahousse, France 2, Géraldine Nakache, |                                |
|     |  Jean-Paul Rouve, Journal de 20 heures   |                                |
|  3  |  Laure Calamy, Bataclan, Novemb

**1. france** - *France*

**2. canada** - *Canada*

**3. playstation** - *PlayStation - Community • Sharing • Help*

**4. Wovenhand** - *Wovenhand*


Top Reddit Posts from Related Subreddits:
+-----+-------------+------------------------------------------+---------+
| No. |  Subreddit  |                  Title                   | Upvotes |
+-----+-------------+------------------------------------------+---------+
|  1  |    france   |      Week-end Création - semaine 45      |    2    |
|  2  |    france   |         Forum Libre - 2024-11-03         |    9    |
|  3  |    france   |        Notepad++ is 21 years old         |   400   |
|  4  |    france   | Romainville : accusé d’avoir simulé son  |   159   |
|     |             |  agression lors d’une conférence sur la  |         |
|     |             |      Palestine, un élu démissionne       |         |
|  5  |    france   | Accident de chasse : un cerf piétine un  |   707   |
|     |             |                 chasseur                 |         |
|  6  |    canada   |  Saturdays and Sundays are now Opinion-  |   109   |
|     |             |                free days           

### Generated Flashy Script for 'Romainville : accusé d’avoir simulé son agression lors d’une conférence sur la Palestine, un élu démissionne':

FADE IN:

[00:00-00:02]
(Stock footage of a bustling cityscape, possibly Parisian, with the text "ROMAINVILLE" appearing in bold, edgy font.)

NARRATOR (V.O.)
"In the heart of Romainville, a political storm is brewing..."

[00:03-00:07]
(Cut to stock footage of a generic politician at a podium, crowds cheering. Text overlays: "ELECTED OFFICIAL.")

NARRATOR (V.O.)
"An elected official, the city's pride, now the subject of a shocking scandal..."

[00:08-00:12]
(Stock footage of a staged fight scene, with the text "THE ACCUSATION" flashing on the screen.)

NARRATOR (V.O.)
"Accused of staging his own assault during a conference on Palestine."

[00:13-00:18]
(Quick cut to stock image of an anonymous Reddit post, comments flowing rapidly.)

NARRATOR (V.O.)
"The incident, now a hot topic on Reddit, stirs a whirlpool of opinions..."

[00:19-00:25]
(Stock footage of a gavel striking, the words "RESIGNATION" flash on screen.)

NARRATOR (V.O.)
"Under the weight of the accusation, the official resigns, leaving the city in shock..."

[00:26-00:32]
(Stock footage of citizens, expressive faces of shock, disbelief, and disappointment.)

NARRATOR (V.O.)
"Romainville citizens, bewildered...betrayed."

[00:33-00:40]
(Cut to stock footage of a person typing on a computer. The words "NEUTRAL SENTIMENT" appear.)

NARRATOR (V.O.)
"Yet on Reddit, the sentiment remains neutral..."

[00:41-00:47]
(Stock footage of a question mark, social media icons, and people in discussion.)

NARRATOR (V.O.)
"Is this a case of political sabotage, or a self-inflicted downfall?"

[00:48-00:55]
(Stock footage of cityscape at sunset, lights flickering on.)

NARRATOR (V.O.)
"As the sun sets on Romainville, the question lingers in the air..."

[00:56-00:60]
(Fade out to black screen with the text: "JOIN THE DISCUSSION".)

NARRATOR (V.O.)
"Join the discussion... What's your take?"

FADE OUT.

NARRATOR (V.O.)
"Stay tuned for updates."

END OF VIDEO.

====


Script execution completed.
